In [1]:
import pandas as pd
import numpy as np
from scipy import stats
from rl687.policies.Linear_Softmax import LinearSoftmax
import csv
from sklearn.model_selection import train_test_split
import cma

In [2]:
class data_preprocess:
    def __init__(self,data_path,num_batches=5):
        self.path=data_path
        self.num_batches=num_batches
        data=[]
        with open(self.path,newline='',encoding='utf-8') as csv_file:
            file=csv.reader(csv_file,delimiter=',')
            for i, H in enumerate(file):
                data.append(H)
        self.data=np.asarray(data)
        self.data_batches=[]
        self.N=200000
        x=self.N//num_batches
        for i in range(num_batches):
            start=i*x
            end=start+x if start+x<=200000 else 200000
            self.data_batches.append(self.data[start:end])
        self.D_s=[]
        self.D_c=[]
        
    def split_data(self,split):
        for i in range(self.num_batches):
            D_c, D_s = train_test_split(self.data_batches[i],test_size=split)
            self.D_c.append(D_c)
            self.D_s.append(D_s)


        
    

In [3]:
class HCOPE():
    def __init__(self,D_c,theta_b,num_actions,num_features,order,n_Ds,delta,lower_bound):
            self.D_c=D_c
            self.theta_b=theta_b
            self.num_actions=num_actions
            self.num_state_features=num_features
            self.order=order
            self.Ds_size=n_Ds
            self.delta=delta
            self.b=lower_bound
            self.pb=LinearSoftmax(self.num_actions,self.order, self.num_state_features,0.001,self.theta_b,sigma=1)
    
    def PDIS(self,H,pe):
        prev_prod=1
        pdis=0
        for i in range(len(H)//3):
            s=np.float(H[i*3])
            #print(i)
            a=int(H[i*3+1])
            r=np.float(H[i*3+2])
            #print(pe.policy(s,a))
            prod=(float(pe.policy(s,a))/float(self.pb.policy(s,a)))*prev_prod
            pdis+=r*prod
            prev_prod=prod
        return pdis
    
    def thetaTopolicy(self,theta):
        p=LinearSoftmax(self.num_actions,self.order, self.num_state_features,0.001,theta,sigma=1)
        #p.theta=theta
        return p
    
    def estimate_J(self,data,theta_e):
        policy_b=self.thetaTopolicy(self.theta_b)
        policy_e=self.thetaTopolicy(theta_e)
        J=0
        pdis_list=[]
        n=data.shape[0]
        for H in data:
            pdis_H=self.PDIS(H,policy_e)
            J+=pdis_H
            pdis_list.append(pdis_H)
        J/=n
        pdis_list=np.array(pdis_list)
        return J,pdis_list
    
    def ssd(self,J,pdis_list):
        n=len(pdis_list)
        return ((np.sum((pdis_list-J)**2)/(n-1))**(0.5))

    def student_t_test(self,D,theta_e):
        J,PDIS_list=self.estimate_J(D,theta_e)
        std=self.ssd(J,PDIS_list)
        n=len(PDIS_list)
        return (J-((std/n**(0.5))*stats.t.ppf(1-self.delta,n-1)))
    
    def optimise_PDIS(self,theta_e):
        J,PDIS_list=self.estimate_J(self.D_c,theta_e)
        std=self.ssd(J,PDIS_list)
        g=J-2*((std/self.Ds_size**(0.5))*stats.t.ppf(1-self.delta,self.Ds_size-1))
        if g<self.b:
            return 10000
        else: return -J
           

In [44]:
def CMAES_policy(D_c, theta_e,theta_b,num_actions,num_features,order,n_Ds,delta,b):
    pdis = HCOPE(D_c,theta_b,num_actions,num_features,order,n_Ds,delta,b)
    c=0
    es = cma.CMAEvolutionStrategy(theta_b, 1.0)
    #ret = es.optimize(pdis.optimise_PDIS)
    while es.stop() or c<30:
        solutions = es.ask()
        es.tell(solutions, [pdis.optimise_PDIS(x) for x in solutions])
        es.logger.add()  # write data to disc to be plotted
        es.disp()
        c+=1
        if c % 4 == 0:
            print(es.result[0])
            #print(func.max())
    print(es.result)
    return es.result

In [45]:
num_a=2
num_dims=1
order=1
theta_e=np.array([1,1,0.01,-0.01])
theta_b=np.array([0.01,-0.01,1,1])
delta=0.04
b=1

In [46]:
data=data_preprocess('data.csv',5)
data.split_data(0.3)

In [47]:

D_c=data.D_c[0]
D_s=data.D_s[0]
n_Ds=len(D_s)

In [ ]:
param=CMAES_policy(D_c, theta_e,theta_b,num_a,num_dims,order,n_Ds,delta,b)

(4_w,8)-aCMA-ES (mu_w=2.6,w_1=52%) in dimension 4 (seed=405296, Fri Dec 13 11:19:24 2019)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1      8 -9.487177892399998e+00 1.0e+00 1.11e+00  1e+00  1e+00 1:22.0
    2     16 -1.211830236689785e+01 1.3e+00 1.48e+00  1e+00  2e+00 2:39.7


In [ ]:
print(param.result_pretty())

In [ ]:
#policies 
policies=[[0.6627, 0.9739, -0.9077, -0.9822],
[1,1,-1,-1],
[1.4950253 , 1.38426051,  -2.05568217, -0.69637852],
[10.22528905, 8.28499955, -0.93434349, -9.96264375],
          
         
         ]

In [ ]:
#test on safety dataset
theta_e=np.asarray([1.4950253 , 1.38426051,  -2.05568217, -0.69637852])
D_s=data.D_s[0]
test=HCOPE(D_c,theta_b,num_a,num_dims,order,n_Ds,0.01,b)
print(test.student_t_test(D_s,theta_e))

In [ ]:
 -1.569038671990480e+01 2.0e+02 1.38e+01  8e+00  3e+01